# Assignment - 05
#### Read Ireland Population data from 
https://data.cso.ie/#
#### In that go to pupulation FY006A
#### Get API Data Query Link to read data in csv
https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en
#### Read data in to CSV from link using pands
#### Dope a column using pands refer document
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html

In [331]:
# import requir library
import pandas as pd

In [332]:
# read the csv file from the URL
df = pd.read_csv('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en')
print(df.head(3))

   STATISTIC Statistic Label  TLIST(A1)  CensusYear C02199V02655         Sex  \
0  FY006AC01      Population       2022        2022            -  Both sexes   
1  FY006AC01      Population       2022        2022            -  Both sexes   
2  FY006AC01      Population       2022        2022            -  Both sexes   

  C02076V03371 Single Year of Age                          C03789V04537  \
0            -           All ages                                   IE0   
1            -           All ages  2ae19629-1492-13a3-e055-000000000001   
2            -           All ages  2ae19629-1433-13a3-e055-000000000001   

  Administrative Counties    UNIT    VALUE  
0                 Ireland  Number  5149139  
1   Carlow County Council  Number    61968  
2     Dublin City Council  Number   592713  


In [333]:
# Save to local file
FileName = 'population.csv'
Location = '../data/'

In [334]:
# Save the dataframe to a CSV file
# This CSV file save to check data locally
df.to_csv(Location + FileName, index=False) 

In [335]:
# Manuplate the data as needed
# Remove unwanted columns
df.drop(columns=['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'C02199V02655', 'C02076V03371', 'C03789V04537', 'UNIT'], inplace=True )
# Filter out rows where 'Single Year of Age' is 'All Ages'
df = df[df['Single Year of Age'] != 'All ages']
# Replace some string of age in to valve
df['Single Year of Age'] = df['Single Year of Age'].replace({'Under 1 year': '0', '100 years and over': '100'})
# Remove non-digit characters and convert to integer
df['Single Year of Age'] = df['Single Year of Age'].replace('\D', '', regex=True)
df['Single Year of Age']=df['Single Year of Age'].astype('int64')

<>:9: SyntaxWarning: invalid escape sequence '\D'
<>:9: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipykernel_28333/3395586782.py:9: SyntaxWarning: invalid escape sequence '\D'
  df['Single Year of Age'] = df['Single Year of Age'].replace('\D', '', regex=True)


In [336]:
# Save the cleaned dataframe to a CSV file for further analysis
df.to_csv(Location + FileName, index=False) 

In [337]:
# Conver data in to the pivot table for better analysis
df_pivot = pd.pivot_table(df,columns=['Administrative Counties'], index = ['Single Year of Age', 'Sex'], values ='VALUE')
print(df_pivot.head(3))
df_pivot.to_csv(Location + 'population_analysis.csv', index=False)

Administrative Counties        Carlow County Council  Cavan County Council  \
Single Year of Age Sex                                                       
0                  Both sexes                  699.0                1006.0   
                   Female                      353.0                 501.0   
                   Male                        346.0                 505.0   

Administrative Counties        Clare County Council  Cork City Council  \
Single Year of Age Sex                                                   
0                  Both sexes                1377.0             2283.0   
                   Female                     691.0             1124.0   
                   Male                       686.0             1159.0   

Administrative Counties        Cork County Council  Donegal County Council  \
Single Year of Age Sex                                                       
0                  Both sexes               4190.0                  1797.0   
    

# END